# Imports

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
import pandas as pd
import pickle
import numpy as np
from sklearn.externals import joblib
from sklearn import *
from sklearn import cluster
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing, cross_validation, neighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Load clean loan csv 

In [30]:
loan_df = pd.read_csv('/Users/madhu/Desktop/ADS Project/Part-1/Loan.csv', low_memory=False) 

In [31]:
loan_df.columns

Index(['id', 'debt_settlement_flag', 'application_type', 'fico_range_low',
       'fico_range_high', 'emp_length', 'dti', 'annual_inc', 'grade',
       'sub_grade', 'int_rate', 'loan_amnt', 'issue_d', 'purpose', 'State',
       'home_ownership', 'zip_code', 'policy_code', 'term', 'Year', 'Month',
       'fico', 'approval'],
      dtype='object')

# Remove General Columns Not Needed

In [32]:
feature_df = loan_df[['application_type','emp_length', 'dti', 'annual_inc',
       'int_rate', 'loan_amnt', 'purpose', 'State', 'home_ownership', 'term', 'fico', 'approval']]

In [33]:
home_ownerships = pd.get_dummies(feature_df['home_ownership'], prefix='house')
feature_df = feature_df.join(home_ownerships)
feature_df.drop('home_ownership', axis=1, inplace=True)

app_type = pd.get_dummies(feature_df['application_type'], prefix='appType')
feature_df = feature_df.join(app_type)
feature_df.drop('application_type', axis=1, inplace=True)

adr_states = pd.get_dummies(feature_df['State'], prefix='s')
feature_df = feature_df.join(adr_states)
feature_df.drop('State', axis=1, inplace=True)

#purpose   
purpose = pd.get_dummies(feature_df['purpose'], prefix='purpose')
feature_df = feature_df.join(purpose)
feature_df.drop('purpose', axis=1, inplace=True)

# No Clustering

In [7]:
zero_cluster_df = feature_df

In [8]:
X_train, X_test, y_train, y_test = train_test_split(zero_cluster_df.ix[:, zero_cluster_df.columns != 'int_rate'], 
                                            zero_cluster_df['int_rate'], 
                                            test_size=0.30)

In [9]:
# Creating the model
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [10]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.500781459962
-----Train-----
RMS:  3.3238853761475706
MAE:  2.54646047597
MAPE:  21.024739445284705
-----Test-----
RMS:  4.089626629741201
MAE:  3.13294503767
MAPE:  25.919863839502433


# Manual Clustering

In [11]:
manual_df = feature_df
def clustering(fico):
    cluster_name = ''
    if fico>790.0:
        cluster_name ='K1'
    elif ((fico <= 790.0) & (fico>750.0)):
        cluster_name = 'K2'
    elif ((fico <=750.0) & (fico>700.0)):
        cluster_name ='K3'
    elif ((fico <=700.0) & (fico>660.0)):
        cluster_name= 'K4'
    return cluster_name
manual_df['cluster'] = manual_df['fico'].astype(float).map(lambda x: clustering(x))

In [12]:
manual_df.head()

,emp_length,dti,annual_inc,int_rate,loan_amnt,term,fico,approval,house_ANY,house_MORTGAGE,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,cluster
0,10,27.65,24000.0,10.65,5000.0,36,737.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
1,1,1.00,30000.0,15.27,2500.0,60,742.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
2,10,8.72,12252.0,15.96,2400.0,36,737.0,1,0,0,...,0,0,0,0,0,0,1,0,0,K3
3,10,20.00,49200.0,13.49,10000.0,36,692.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4
4,1,17.94,80000.0,12.69,3000.0,60,697.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4


In [13]:
#Seperate into diff dataframes
manual_c1 = manual_df[manual_df['cluster']=='K1']
manual_c1.drop('cluster',axis=1,inplace=True)
manual_c2 = manual_df[manual_df['cluster']=='K2']
manual_c2.drop('cluster',axis=1,inplace=True)
manual_c3 = manual_df[manual_df['cluster']=='K3']
manual_c3.drop('cluster',axis=1,inplace=True)
manual_c4 = manual_df[manual_df['cluster']=='K4']
manual_c4.drop('cluster',axis=1,inplace=True)

#### Manual 1

In [14]:

X_train, X_test, y_train, y_test = train_test_split(manual_c1.ix[:, manual_c1.columns != 'int_rate'], 
                                            manual_c1['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [15]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.423053507183
-----Train-----
RMS:  2.3832303376179427
MAE:  1.66990460776
MAPE:  19.916846779674614
-----Test-----
RMS:  3.0006347344412063
MAE:  2.10985909842
MAPE:  24.98023761580151


#### Manual 2

In [16]:
#manual 2
X_train, X_test, y_train, y_test = train_test_split(manual_c2.ix[:, manual_c2.columns != 'int_rate'], 
                                            manual_c2['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [17]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.410091071279
-----Train-----
RMS:  2.7745046713217634
MAE:  2.00997510905
MAPE:  22.05226303467547
-----Test-----
RMS:  3.495193765763599
MAE:  2.50564200562
MAPE:  27.210155312679124


#### Manual 3

In [18]:
#Manual 3
X_train, X_test, y_train, y_test = train_test_split(manual_c3.ix[:, manual_c3.columns != 'int_rate'], 
                                            manual_c3['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [19]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.44090119308
-----Train-----
RMS:  3.1984852247222966
MAE:  2.44187532688
MAPE:  22.36565676848448
-----Test-----
RMS:  3.8992646454444926
MAE:  2.97721509419
MAPE:  27.40320718379235


#### Manual 4

In [20]:
#Manual 4
X_train, X_test, y_train, y_test = train_test_split(manual_c4.ix[:, manual_c4.columns != 'int_rate'], 
                                            manual_c4['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [21]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.427644147724
-----Train-----
RMS:  3.406473266675882
MAE:  2.62916904213
MAPE:  19.50449877777428
-----Test-----
RMS:  4.174935617126658
MAE:  3.22300243768
MAPE:  23.931813923025302


# K-Means

In [34]:
k_cluster = joblib.load(open('/Users/madhu/Desktop/ADS Project/Part-2/Clustering/kmeanCluster.pkl', 'rb'))
#Create new column in load_df for the k-means clustering 
k_mean_feature_df = feature_df
k_mean_feature_df['kMean'] = k_cluster.labels_

In [35]:
kmean_0 = k_mean_feature_df[k_mean_feature_df['kMean']==0]
kmean_0.drop('kMean', axis=1, inplace=True)
kmean_1 = k_mean_feature_df[k_mean_feature_df['kMean']==1]
kmean_1.drop('kMean', axis=1, inplace=True)
kmean_2 = k_mean_feature_df[k_mean_feature_df['kMean']==2]
kmean_2.drop('kMean', axis=1, inplace=True)
kmean_3 = k_mean_feature_df[k_mean_feature_df['kMean']==3]
kmean_3.drop('kMean', axis=1, inplace=True)
kmean_4 = k_mean_feature_df[k_mean_feature_df['kMean']==4]
kmean_4.drop('kMean', axis=1, inplace=True)
kmean_5 = k_mean_feature_df[k_mean_feature_df['kMean']==5]
kmean_5.drop('kMean', axis=1, inplace=True)
kmean_6 = k_mean_feature_df[k_mean_feature_df['kMean']==6]
kmean_6.drop('kMean', axis=1, inplace=True)
kmean_7 = k_mean_feature_df[k_mean_feature_df['kMean']==7]
kmean_7.drop('kMean', axis=1, inplace=True)


#### K-means0

In [37]:
#K-mean_0
X_train, X_test, y_train, y_test = train_test_split(kmean_0.ix[:, kmean_0.columns != 'int_rate'], 
                                            kmean_0['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [38]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.46457844244
-----Train-----
RMS:  3.163208792383198
MAE:  2.41684900598
MAPE:  21.15392314319859
-----Test-----
RMS:  3.8667021784880786
MAE:  2.9562109764
MAPE:  26.073330848309578


#### K-means1

In [51]:
#K-mean_1
X_train, X_test, y_train, y_test = train_test_split(kmean_1.ix[:, kmean_1.columns != 'int_rate'], 
                                            kmean_1['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [12]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.483736198004
-----Train-----
RMS:  3.5459357066693404
MAE:  2.73329479989
MAPE:  19.768568426856
-----Test-----
RMS:  4.377867964260734
MAE:  3.38174637409
MAPE:  24.54556532824701


#### K-means2

In [39]:
#K-mean_2
X_train, X_test, y_train, y_test = train_test_split(kmean_2.ix[:, kmean_2.columns != 'int_rate'], 
                                            kmean_2['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [40]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.479224104241
-----Train-----
RMS:  3.4784170331310693
MAE:  2.67286501533
MAPE:  20.605555159249057
-----Test-----
RMS:  4.256611063001298
MAE:  3.27708997855
MAPE:  25.479751837858284


#### K-means3

In [41]:
#K-mean_3
X_train, X_test, y_train, y_test = train_test_split(kmean_3.ix[:, kmean_3.columns != 'int_rate'], 
                                            kmean_3['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [42]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.397915297098
-----Train-----
RMS:  3.2861409674232136
MAE:  2.53208446698
MAPE:  22.89197624777424
-----Test-----
RMS:  3.9945055186834293
MAE:  3.07377908308
MAPE:  27.842917356251284


#### K-means4

In [43]:
#K-mean_4
X_train, X_test, y_train, y_test = train_test_split(kmean_4.ix[:, kmean_4.columns != 'int_rate'], 
                                            kmean_4['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [44]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.293537543602
-----Train-----
RMS:  4.479558938892795
MAE:  3.45644452223
MAPE:  24.393988095689576
-----Test-----
RMS:  5.7412177401307325
MAE:  4.38395102041
MAPE:  30.972916053413623


#### K-means5

In [45]:
#K-mean_5
X_train, X_test, y_train, y_test = train_test_split(kmean_5.ix[:, kmean_5.columns != 'int_rate'], 
                                            kmean_5['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [46]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.44491475578
-----Train-----
RMS:  3.6448698740782386
MAE:  2.80613695277
MAPE:  19.539661012433974
-----Test-----
RMS:  4.418021373501154
MAE:  3.40123447238
MAPE:  23.769021165764666


#### K-mean_6

In [47]:
#K-mean_6
X_train, X_test, y_train, y_test = train_test_split(kmean_6.ix[:, kmean_6.columns != 'int_rate'], 
                                            kmean_6['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [48]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.476204604049
-----Train-----
RMS:  3.279912741562528
MAE:  2.50715632301
MAPE:  20.861852490342496
-----Test-----
RMS:  4.022961281370181
MAE:  3.07546478742
MAPE:  25.78431650856751


#### K-means7

In [49]:
#K-mean_7
X_train, X_test, y_train, y_test = train_test_split(kmean_7.ix[:, kmean_7.columns != 'int_rate'], 
                                            kmean_7['int_rate'], 
                                            test_size=0.30)
clf = neighbors.KNeighborsRegressor()
clf.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

In [50]:
# Accuracy of the model created
accuracy = clf.score(X_train, y_train)
print(accuracy)

y_test_predicted = clf.predict(X_test)
y_train_predicted= clf.predict(X_train)    

#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("-----Train-----")
print('RMS: ',rms)
print('MAE: ',mae_train)
print('MAPE: ',mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("-----Test-----")
print('RMS: ',rms)
print('MAE: ',mae_test)
print('MAPE: ',mape_test)

0.442103614384
-----Train-----
RMS:  3.102149168515197
MAE:  2.37596675236
MAPE:  21.653252528237203
-----Test-----
RMS:  3.8231429610351237
MAE:  2.92680114308
MAPE:  26.738903525497363


The text in the document by SHREYANTH REDDY BEZAWADA is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/

Copyright 2018 SHREYANTH REDDY BEZAWADA AND ADITYA MOHAN KUMAR

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.